In [1]:
#Import Main Dependencies
import pandas as pd
from sqlalchemy import create_engine
from unicodedata import normalize

In [2]:
# #Read in the emissions data file 
# test_file = "resources/test.csv"
# test_df = pd.read_csv(test_file)
# test_df.head()

In [3]:
#Read in the emissions data file 
transactions_file = "resources/transactions.csv"
trans_df = pd.read_csv(transactions_file)
trans_df.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [4]:
# check data types for SQL compatibility 
trans_df.dtypes

date            object
store_nbr        int64
transactions     int64
dtype: object

In [5]:
trans_df['date'] = pd.to_datetime(trans_df['date'])
trans_df.dtypes

date            datetime64[ns]
store_nbr                int64
transactions             int64
dtype: object

In [6]:
# earthquake = (trans_df['date'] >= '2016-04-16') & (trans_df['date'] < '2016-05-16')
# # mask = (df['date'] > start_date) & (df['date'] <= end_date)
# eq_df = trans_df.loc[earthquake]
# eq_df.head()
# eq_df.tail()
# len(eq_df)

In [7]:
april_df = trans_df[((trans_df['date'] >= '2013-04-16') & (trans_df['date'] < '2013-05-16'))
    | ((trans_df['date'] >= '2014-04-16') & (trans_df['date'] < '2014-05-16'))
    | ((trans_df['date'] >= '2015-04-16') & (trans_df['date'] < '2015-05-16'))
    | ((trans_df['date'] >= '2016-04-16') & (trans_df['date'] < '2016-05-16'))
    | ((trans_df['date'] >= '2017-04-16') & (trans_df['date'] < '2017-05-16'))]
april_df.head()

,date,store_nbr,transactions
4785,2013-04-16,1,1757
4786,2013-04-16,2,1742
4787,2013-04-16,3,2813
4788,2013-04-16,4,1400
4789,2013-04-16,5,1565


In [8]:
april_df.tail()

,date,store_nbr,transactions
78515,2017-05-15,50,2416
78516,2017-05-15,51,1525
78517,2017-05-15,52,1896
78518,2017-05-15,53,1013
78519,2017-05-15,54,884


In [9]:
len(april_df)

7443

In [10]:
# test = (trans_df['date'] >= '2017-04-16') & (trans_df['date'] < '2017-05-16')
# test_df = trans_df.loc[test]
# test_df.head()
# test_df.tail()
# len(test_df)

In [11]:
#Read in the emissions data file 
stores_file = "resources/stores.csv"
stores_df = pd.read_csv(stores_file)
stores_df.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [12]:
stores_df.dtypes

store_nbr     int64
city         object
state        object
type         object
cluster       int64
dtype: object

In [13]:
store_april = pd.merge(stores_df, april_df, on ='store_nbr')
store_april.head()

,store_nbr,city,state,type,cluster,date,transactions
0,1,Quito,Pichincha,D,13,2013-04-16,1757
1,1,Quito,Pichincha,D,13,2013-04-17,1807
2,1,Quito,Pichincha,D,13,2013-04-18,1607
3,1,Quito,Pichincha,D,13,2013-04-19,1737
4,1,Quito,Pichincha,D,13,2013-04-20,1335


In [14]:
len(store_april)

7443

In [15]:
#Read in the emissions data file 
holiday_file = "resources/holidays_events.csv"
holiday_df = pd.read_csv(holiday_file)
holiday_df.tail()

,date,type,locale,locale_name,description,transferred
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False
349,2017-12-26,Additional,National,Ecuador,Navidad+1,False


In [16]:
len(holiday_df)

350

In [17]:
holiday_df.dtypes

date           object
type           object
locale         object
locale_name    object
description    object
transferred      bool
dtype: object

In [18]:
holiday_df['date'] = pd.to_datetime(holiday_df['date'])
holiday_df.dtypes

date           datetime64[ns]
type                   object
locale                 object
locale_name            object
description            object
transferred              bool
dtype: object

In [19]:
#drop rows that contain specific 'value' in 'column_name'
cleaned_hol = holiday_df[holiday_df['transferred'] == False]

In [20]:
# cleaned_holiday= holiday_df.loc[holiday_df["transferred"] != "FALSE"]
len(cleaned_hol)

338

In [21]:
# store_april.drop_duplicates(subset='date', inplace=True)
# len(store_april)

In [22]:
cleaned_hol.drop_duplicates(subset='date', inplace=True)
len(cleaned_hol)

/var/folders/bp/kkf4c1nj557c5frmyn420c7c0000gn/T/ipykernel_34789/415807744.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_hol.drop_duplicates(subset='date', inplace=True)


300

In [23]:
merged_holiday = pd.merge(store_april, cleaned_hol, on ='date', how="left")
merged_holiday.head()

,store_nbr,city,state,type_x,cluster,date,transactions,type_y,locale,locale_name,description,transferred
0,1,Quito,Pichincha,D,13,2013-04-16,1757,NaN,NaN,NaN,NaN,NaN
1,1,Quito,Pichincha,D,13,2013-04-17,1807,NaN,NaN,NaN,NaN,NaN
2,1,Quito,Pichincha,D,13,2013-04-18,1607,NaN,NaN,NaN,NaN,NaN
3,1,Quito,Pichincha,D,13,2013-04-19,1737,NaN,NaN,NaN,NaN,NaN
4,1,Quito,Pichincha,D,13,2013-04-20,1335,NaN,NaN,NaN,NaN,NaN


In [24]:
# trans_store_hol = pd.concat([store_april, indexed_hol], axis=1)
# trans_store_hol = pd.concat([store_april,indexed_hol],axis=1,join='outer')
len(merged_holiday)

7443

In [25]:
merged_holiday.head()

,store_nbr,city,state,type_x,cluster,date,transactions,type_y,locale,locale_name,description,transferred
0,1,Quito,Pichincha,D,13,2013-04-16,1757,NaN,NaN,NaN,NaN,NaN
1,1,Quito,Pichincha,D,13,2013-04-17,1807,NaN,NaN,NaN,NaN,NaN
2,1,Quito,Pichincha,D,13,2013-04-18,1607,NaN,NaN,NaN,NaN,NaN
3,1,Quito,Pichincha,D,13,2013-04-19,1737,NaN,NaN,NaN,NaN,NaN
4,1,Quito,Pichincha,D,13,2013-04-20,1335,NaN,NaN,NaN,NaN,NaN


In [26]:
merged_holiday.tail()

,store_nbr,city,state,type_x,cluster,date,transactions,type_y,locale,locale_name,description,transferred
7438,54,El Carmen,Manabi,C,3,2017-05-11,747,NaN,NaN,NaN,NaN,NaN
7439,54,El Carmen,Manabi,C,3,2017-05-12,735,Holiday,Local,Puyo,Cantonizacion del Puyo,False
7440,54,El Carmen,Manabi,C,3,2017-05-13,1112,Additional,National,Ecuador,Dia de la Madre-1,False
7441,54,El Carmen,Manabi,C,3,2017-05-14,1212,Event,National,Ecuador,Dia de la Madre,False
7442,54,El Carmen,Manabi,C,3,2017-05-15,884,NaN,NaN,NaN,NaN,NaN


In [27]:
#Read in the emissions data file 
oil_file = "resources/oil.csv"
oil_df = pd.read_csv(oil_file)
oil_df.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [28]:
# drop null values from oil dataframe/csv
cleaned_oil=oil_df.dropna()
cleaned_oil.head()

,date,dcoilwtico
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
5,2013-01-08,93.21


In [29]:
cleaned_oil['date'] = pd.to_datetime(cleaned_oil['date'])
cleaned_oil.dtypes

/var/folders/bp/kkf4c1nj557c5frmyn420c7c0000gn/T/ipykernel_34789/3394931503.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_oil['date'] = pd.to_datetime(cleaned_oil['date'])


date          datetime64[ns]
dcoilwtico           float64
dtype: object

In [30]:
# cleaned_hol.squeeze()
indexed_oil = cleaned_oil.set_index('date', inplace=True)

In [32]:
cleaned_df = pd.concat([merged_holiday, indexed_oil], axis=1)
len(cleaned_df)

7443

In [33]:
# cleaned_df = pd.merge(trans_store_hol, cleaned_oil, on ='date')
cleaned_df.head()

,store_nbr,city,state,type_x,cluster,date,transactions,type_y,locale,locale_name,description,transferred
0,1,Quito,Pichincha,D,13,2013-04-16,1757,NaN,NaN,NaN,NaN,NaN
1,1,Quito,Pichincha,D,13,2013-04-17,1807,NaN,NaN,NaN,NaN,NaN
2,1,Quito,Pichincha,D,13,2013-04-18,1607,NaN,NaN,NaN,NaN,NaN
3,1,Quito,Pichincha,D,13,2013-04-19,1737,NaN,NaN,NaN,NaN,NaN
4,1,Quito,Pichincha,D,13,2013-04-20,1335,NaN,NaN,NaN,NaN,NaN


In [35]:
# drop transferred, cluster and type_x column 
final_df = cleaned_df.drop(['type_x', "cluster", "transferred" ], axis=1)
final_df.head()

,store_nbr,city,state,date,transactions,type_y,locale,locale_name,description
0,1,Quito,Pichincha,2013-04-16,1757,NaN,NaN,NaN,NaN
1,1,Quito,Pichincha,2013-04-17,1807,NaN,NaN,NaN,NaN
2,1,Quito,Pichincha,2013-04-18,1607,NaN,NaN,NaN,NaN
3,1,Quito,Pichincha,2013-04-19,1737,NaN,NaN,NaN,NaN
4,1,Quito,Pichincha,2013-04-20,1335,NaN,NaN,NaN,NaN


In [36]:
final_df.dtypes

store_nbr                int64
city                    object
state                   object
date            datetime64[ns]
transactions             int64
type_y                  object
locale                  object
locale_name             object
description             object
dtype: object

In [37]:
len(final_df)

7443

In [38]:
#Create SQL Alchemy engine to postgres database "IMDB_db"
protocol = 'postgresql'
username = 'postgres'
password = '0206Teddy'
host = 'localhost'
port = 5432
database_name = 'grocery'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [39]:
#show the existing table names in the database
engine.table_names()

/var/folders/bp/kkf4c1nj557c5frmyn420c7c0000gn/T/ipykernel_34789/2549929793.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['earthquake_data']

In [40]:
#write the data in the dataframe to the database table
final_df.to_sql(name='earthquake_data', con=engine, if_exists='replace', index=False)

443

In [41]:
#Query the carbon emissions data table to confirm the above data load
pd.read_sql_query('select * from public."earthquake_data"', con=engine)

,store_nbr,city,state,date,transactions,type_y,locale,locale_name,description
0,1,Quito,Pichincha,2013-04-16,1757,None,None,None,None
1,1,Quito,Pichincha,2013-04-17,1807,None,None,None,None
2,1,Quito,Pichincha,2013-04-18,1607,None,None,None,None
3,1,Quito,Pichincha,2013-04-19,1737,None,None,None,None
4,1,Quito,Pichincha,2013-04-20,1335,None,None,None,None
...,...,...,...,...,...,...,...,...,...
7438,54,El Carmen,Manabi,2017-05-11,747,None,None,None,None
7439,54,El Carmen,Manabi,2017-05-12,735,Holiday,Local,Puyo,Cantonizacion del Puyo
7440,54,El Carmen,Manabi,2017-05-13,1112,Additional,National,Ecuador,Dia de la Madre-1
7441,54,El Carmen,Manabi,2017-05-14,1212,Event,National,Ecuador,Dia de la Madre


In [42]:
pd.read_sql_query('select * from public."earthquake_data"', con=engine).count()

store_nbr       7443
city            7443
state           7443
date            7443
transactions    7443
type_y          2613
locale          2613
locale_name     2613
description     2613
dtype: int64